In [120]:
##### importing custom modules from the projects folder
import sys, os
from pathlib import Path
# Add project root to sys.path - search backwards through folders to find config.py
cwd = Path.cwd()
# Search upwards until a "config*" file is found
for parent in [cwd, *cwd.parents]:
    match = next(parent.glob('config*'), None)
    if match:
        PROJECT_ROOT = match.parent
        break
sys.path.append(str(PROJECT_ROOT))
import modules.helperModule as hf
##### -------------------------------------------------
from datetime import date
import pandas as pd

In [121]:
dfDrafts = hf.query_database(
    'SELECT * FROM ktbdrafts;'
)
dfProj = hf.query_database(
    """
    SELECT player.name, pos.pos, p.* 
    FROM projection p
    INNER JOIN player ON p.playerId = player.playerId
    LEFT JOIN pos ON player.posId = pos.posId
    WHERE season = 2025
    """
)
dfRank = hf.query_database(
    """
    SELECT player.name, pos.pos, o.outletName, a.analystName, r.* 
    FROM ranking r
    INNER JOIN player ON r.playerId = player.playerId
    LEFT JOIN pos ON player.posId = pos.posId
    LEFT JOIN outlet o ON r.outletId = o.outletId
    LEFT JOIN analyst a ON r.analystId = a.analystID
    WHERE season = 2025;
    """
)
dfAdp= hf.query_database(
    """
    SELECT player.name, pos.pos, o.outletName, a.* 
    FROM adp a
    INNER JOIN player ON a.playerId = player.playerId
    LEFT JOIN pos ON player.posId = pos.posId
    LEFT JOIN outlet o ON a.outletId = o.outletId
    WHERE date > '2025-04-30';
    """
)

dfPlayers2024 = hf.query_database(
    """
    SELECT kp.playerId, kp.playerName, pos.pos, kp.adp, kp.points, kp.pointsAvg,  
    kp.points/kp.pointsAvg gp, kp.positionRank, kp.overallRank
    FROM ktbplayers kp
    LEFT JOIN player p ON kp.playerId = p.espnId 
    LEFT JOIN pos ON p.posId = pos.posId
    WHERE season = 2024 AND points > 0;
    """
).drop_duplicates(keep='first')

dfPlayers = hf.query_database(
    """
    SELECT player.playerId, player.name, pos.pos 
    FROM player
    LEFT JOIN pos ON pos.posId = player.posId
    WHERE player.playerId IS NOT NULL and player.name IS NOT NULL;
    """
).drop_duplicates(keep='first')

dfProps = hf.query_database(
    """
    SELECT *
    FROM odds_season_totals
    WHERE date = (SELECT MAX(date) FROM odds_season_totals);
    """
)

query successful
query successful
query successful
query successful
query successful
query successful
query successful


<head> VBD CALCS </head>

<ul>
<li>VOLS: replacement = first round after starters can be filled. 9-starter roster = 10th round, so 12 team league would be pick 121.</li>
<li>VORP: replacement = players not drafted so players ranked after draft</li>
<li>Man Games: </li>
<li>Pos by Starter: number of startes at position * number of teams. 1QB, 2RB, 3WR 12 team = QB replacement at 12th ranked, rb at 24th, wr at 36th</li>
<li>historical league drafting habits</li>
<li></li>


</ul>



In [122]:
import pandas as pd

# parameters
pick_cutoff = 120
season_cutoff = 2021
n_teams = 12
needed_starters = {'WR': 2, 'RB': 2, 'QB': 1, 'TE': 1, 'FLEX': 1}
needed_starters = {k: v * n_teams for k, v in needed_starters.items()}

# ---- flex logic
flex_eligible = dfPlayers2024.query(
    "(pos == 'WR' & positionRank > @needed_starters['WR']) | "
    "(pos == 'RB' & positionRank > @needed_starters['RB']) | "
    "(pos == 'TE' & positionRank > @needed_starters['TE'])"
)
flex_starters = (
    flex_eligible.nlargest(needed_starters['FLEX'], 'points')['pos']
    .value_counts()
    .to_dict()
)
for k, v in flex_starters.items():
    needed_starters[k] += v

df_needed_starters = pd.DataFrame(
    list(needed_starters.items()), columns=['pos', 'needed_starters']
)

# ---- helper to compute averages
def avg_counts(df, cutoff_query, col_name):
    counts = (
        df.query(cutoff_query)
        .groupby(['season', 'pos']).size()
        .groupby('pos').mean().astype(int)
        .rename(col_name)
    )
    return counts.reset_index()

avg_all = avg_counts(dfDrafts, f'overallPick <= {pick_cutoff}', 'top_picks_all_years')
avg_last3 = avg_counts(
    dfDrafts, f'overallPick <= {pick_cutoff} & season >= {season_cutoff}',
    'top_picks_last_3_years'
)

# ---- merge summary
pos_keep = ['QB', 'RB', 'WR', 'TE']
avg_pos_counts = (
    avg_all.merge(avg_last3, on='pos', how='outer')
    .merge(df_needed_starters, on='pos', how='outer')
    .fillna(0)
    .query('pos in @pos_keep')
    .sort_values('top_picks_all_years', ascending=False)
    .reset_index(drop=True)
)

# ---- replacement scores
def lookup_score(row, col):
    rank = int(row[col])
    match = dfPlayers2024.query("pos == @row['pos'] & positionRank == @rank")['points']
    return match.iloc[0] if not match.empty else 0

for col in avg_pos_counts.columns[1:]:
    avg_pos_counts[f'{col}_baseline_score'] = avg_pos_counts.apply(
        lambda r: lookup_score(r, col), axis=1
    )

avg_pos_counts


,pos,top_picks_all_years,top_picks_last_3_years,needed_starters,top_picks_all_years_baseline_score,top_picks_last_3_years_baseline_score,needed_starters_baseline_score
0,WR,43.0,46.0,33.0,135.30,127.70,161.70
1,RB,40.0,38.0,27.0,110.10,110.60,162.20
2,QB,13.0,13.0,12.0,280.36,280.36,296.52
3,TE,11.0,11.0,12.0,120.50,120.50,116.10


In [ ]:
# =======================
#     PREP RANKINGS
# =======================

# updating rank groups to standardize position labels
dfRank['rankGroup'] = dfRank['rankGroup'].replace({'DEF':'DST'})

# outletName = fantasyPros ecr ranking is across all positions, have to remove it when aggregating 
# unless its updated to be positional
## updating ECR TO Be positional ranking
mask = dfRank['outletName'] == 'fantasyPros'

dfRank.loc[mask, 'ranking'] = (
    dfRank[mask]
    .groupby('rankGroup')['ranking']
    .rank(method='dense', ascending=True)
)

df_agg_ranking = dfRank.groupby(
    ['name', 'playerId', 'rankGroup', 'date'] 
).agg(
    ranking_mean=('ranking', 'mean')
).reset_index()

# filter to most recent ranking
df_agg_ranking = df_agg_ranking.query('date == date.max()')

# =======================
#     PREP PROJECTIONS
# =======================
# filter to most recent projection
df_agg_projection = dfProj.query('date == date.max() & fantasyPoints > 0')

# calculate agg projections
df_agg_projection = df_agg_projection.groupby(
    ['name', 'playerId', 'date'] 
).agg(
    projections_mean=('fantasyPoints', 'mean')
).reset_index()

# =======================
#     PREP ADP
# =======================
# filter to most recent projection
df_agg_adp = dfAdp.query('date == date.max() & adp > 0')

# calculate agg projections
df_agg_adp = df_agg_adp.groupby(
    ['name', 'playerId', 'date']#, 'outletName'] 
).agg(
    adp_mean=('adp', 'mean'),
).reset_index()

# =======================
#     PREP PROPS
# =======================
# filter to most recent projection
df_agg_props = dfProps.query('date == date.max()')

# calculate agg projections
df_agg_props  = (
    df_agg_props.pivot_table(
        index=['name', 'playerId'],
        columns='prop',
        values='current_line',
        aggfunc='first'   # handles duplicates safely
    )
    .add_suffix('_current_line')   # rename columns
    .reset_index()
)
df_agg_props = df_agg_props.drop(['name'], axis=1)


# =======================
#     COMBINE TO SINGLE RECORDS
# =======================
df_allstats_agg = df_agg_ranking[['playerId', 'ranking_mean']].merge(
    df_agg_projection[['playerId', 'projections_mean']],
    on='playerId',
    how='left'
).merge(
    df_agg_adp[['playerId', 'adp_mean']],
    on='playerId',
    how='left'
).merge(
    df_agg_props,
    on='playerId',
    how='left'
)

df_allstats_agg = df_allstats_agg.merge(
    dfPlayers[['playerId', 'name', 'pos']],
    on='playerId',
    how='left'
)
# =======================
#     aDD ON VBD VALUES
# =======================
df_allstats_agg = df_allstats_agg.merge(
    avg_pos_counts[['pos', 'top_picks_last_3_years_baseline_score',	'needed_starters_baseline_score']],
    how='left',
    on='pos'
)
df_allstats_agg = df_allstats_agg.assign(
    VBD_historical = df_allstats_agg['projections_mean'] - df_allstats_agg['top_picks_last_3_years_baseline_score'],
    VBD_VOLS       = df_allstats_agg['projections_mean'] - df_allstats_agg['needed_starters_baseline_score']
)

In [131]:
col_order = [
    'name', 'pos', 'VBD_historical', 'VBD_VOLS', 'adp_mean', 'ranking_mean', 'projections_mean',  
    'total_passing_tds_current_line', 'total_passing_yds_current_line', 
    'total_receiving_tds_current_line', 'total_receiving_yds_current_line', 
    'total_rushing_tds_current_line', 'total_rushing_yds_current_line', 
    'top_picks_last_3_years_baseline_score', 'needed_starters_baseline_score', 
    'playerId'
]
df_allstats_agg = df_allstats_agg[col_order]
a = df_allstats_agg[df_allstats_agg['pos']=='RB'].sort_values('VBD_VOLS', ascending=False)
a.head(20)

,name,pos,VBD_historical,VBD_VOLS,adp_mean,ranking_mean,projections_mean,total_passing_tds_current_line,total_passing_yds_current_line,total_receiving_tds_current_line,total_receiving_yds_current_line,total_rushing_tds_current_line,total_rushing_yds_current_line,top_picks_last_3_years_baseline_score,needed_starters_baseline_score,playerId
58,Bijan Robinson,RB,200.9,149.3,2.500000,1.000000,311.5,NaN,NaN,2.5,375.5,10.5,1200.5,110.6,162.2,15827
349,Jahmyr Gibbs,RB,198.9,147.3,4.000000,2.200000,309.5,NaN,NaN,NaN,425.5,10.5,1075.5,110.6,162.2,15895
703,Saquon Barkley,RB,195.4,143.8,3.000000,3.200000,306.0,NaN,NaN,1.5,275.5,10.5,1400.5,110.6,162.2,918
459,Josh Jacobs,RB,163.9,112.3,15.500000,10.200000,274.5,NaN,NaN,0.5,274.5,9.5,1050.5,110.6,162.2,765
226,Derrick Henry,RB,160.4,108.8,9.333333,6.500000,271.0,NaN,NaN,0.5,155.5,13.5,1325.5,110.6,162.2,1106
153,Christian McCaffrey,RB,158.9,107.3,8.500000,5.100000,269.5,NaN,NaN,3.5,425.5,7.5,950.5,110.6,162.2,107
44,Ashton Jeanty,RB,152.9,101.3,11.500000,4.666667,263.5,NaN,NaN,NaN,NaN,7.5,1050.5,110.6,162.2,15823
210,De'Von Achane,RB,150.9,99.3,18.500000,6.111111,261.5,NaN,NaN,3.5,450.5,7.5,875.5,110.6,162.2,15860
440,Jonathan Taylor,RB,140.9,89.3,18.166667,8.600000,251.5,NaN,NaN,NaN,175.5,9.5,1175.5,110.6,162.2,616
531,Kyren Williams,RB,139.4,87.8,24.166667,12.200000,250.0,NaN,NaN,NaN,NaN,10.5,1000.5,110.6,162.2,733


In [132]:
df_allstats_agg.to_excel('2025 Draft VBDs.xlsx')

In [133]:
import dash

ModuleNotFoundError: No module named 'dash'